<a href="https://colab.research.google.com/github/MayerT1/COLAB_DEV_GEDI-BDL/blob/main/Neon_tree_height_and_Lidar_CHM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.neonscience.org/resources/learning-hub/tutorials/tree-heights-veg-structure-chm

Make sure this is set up Runtime-> Change Runtime Type -> Run Time Type -> R

# 1. Setup

In [9]:
install.packages("neonUtilities")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’, ‘downloader’, ‘R.utils’, ‘pbapply’




In [10]:
install.packages("neonOS")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [11]:
install.packages("terra")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [12]:
install.packages("devtools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [13]:
devtools::install_github("NEONScience/NEON-geolocation/geoNEON")

R6   (2.6.0 -> 2.6.1) [CRAN]
plyr (NA    -> 1.8.9) [CRAN]


Installing 2 packages: R6, plyr

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmpff3CZd/remotesb55fbba142/NEONScience-NEON-geolocation-59bdef7/geoNEON/DESCRIPTION’ ... OK
* preparing ‘geoNEON’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘geoNEON_1.0.0.9600.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Libraries

In [14]:
library(terra)

terra 1.8.21



In [15]:
library(neonUtilities)

In [16]:
library(neonOS)

In [17]:
library(geoNEON)

In [18]:
options(stringsAsFactors=F)

# set working directory

# adapt directory path for your system

# wd <- "~/data"

# setwd(wd)

In [19]:
wd <- "~/data"


In [23]:
getwd()

[1] "/content"

In [21]:
#setwd(wd)

ERROR: Error in setwd(wd): cannot change working directory


# 2. Vegetation structure data

[Cheatsheet](https://www.neonscience.org/sites/default/files/cheat-sheet-neonUtilities.pdf)

In [22]:
veglist <- loadByProduct(dpID="DP1.10098.001",
                         site="WREF",
                         package="basic",
                         release="RELEASE-2023",
                         check.size = FALSE)

Finding available files
  |======================================================================| 100%

  |======================================================================| 100%

Unpacking zip files using 1 cores.
Stacking operation across a single core.
Stacking table vst_apparentindividual
Stacking table vst_mappingandtagging
Stacking table vst_perplotperyear
Stacking table vst_non-woody
Copied the most recent publication of validation file to /stackedFiles
Copied the most recent publication of categoricalCodes file to /stackedFiles
Copied the most recent publication of variable definition file to /stackedFiles
Finished: Stacked 4 data tables and 4 metadata tables!
Stacking took 2.838805 secs


In [ ]:
head(veglist)

In [ ]:
attributes(veglist)

In [ ]:
vegmap <- getLocTOS(veglist$vst_mappingandtagging,
                          "vst_mappingandtagging")

In [ ]:
head(vegmap)

In [ ]:
attributes(vegmap)

In [ ]:
veg <- joinTableNEON(veglist$vst_apparentindividual,
                     vegmap,
                     name1="vst_apparentindividual",
                     name2="vst_mappingandtagging")

In [ ]:
head(veg)

In [ ]:
attributes(veg)

In [ ]:
symbols(veg$adjEasting[which(veg$plotID=="WREF_075")],
        veg$adjNorthing[which(veg$plotID=="WREF_075")],
        circles=veg$stemDiameter[which(veg$plotID=="WREF_075")]/100/2,
        inches=F, xlab="Easting", ylab="Northing")

In [ ]:
symbols(veg$adjEasting[which(veg$plotID=="WREF_075")],
        veg$adjNorthing[which(veg$plotID=="WREF_075")],
        circles=veg$stemDiameter[which(veg$plotID=="WREF_075")]/100/2,
        inches=F, xlab="Easting", ylab="Northing")

symbols(veg$adjEasting[which(veg$plotID=="WREF_075")],
        veg$adjNorthing[which(veg$plotID=="WREF_075")],
        circles=veg$adjCoordinateUncertainty[which(veg$plotID=="WREF_075")],
        inches=F, add=T, fg="lightblue")

# Canopy height model data

In [ ]:
byTileAOP(dpID="DP3.30015.001", site="WREF", year="2017",
          easting=veg$adjEasting[which(veg$plotID=="WREF_075")],
          northing=veg$adjNorthing[which(veg$plotID=="WREF_075")],
          check.size=FALSE, savepath=wd)



chm <- rast(paste0(wd, "/DP3.30015.001/neon-aop-products/2017/FullSite/D16/2017_WREF_1/L3/DiscreteLidar/CanopyHeightModelGtif/NEON_D16_WREF_DP3_580000_5075000_CHM.tif"))

In [ ]:
plot(chm, col=topo.colors(5))

# 4. Comparing the two datasets

In [ ]:
vegsub <- veg[which(veg$adjEasting >= ext(chm)[1] &
                      veg$adjEasting <= ext(chm)[2] &
                      veg$adjNorthing >= ext(chm)[3] &
                      veg$adjNorthing <= ext(chm)[4]),]

In [ ]:
valCHM <- extract(chm,
                  cbind(vegsub$adjEasting,
                  vegsub$adjNorthing))



plot(valCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegsub$height, pch=20, xlab="Height",
     ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(valCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegsub$height, use="complete")

## [1] 0.3824467

In [ ]:
valCHMbuff <- extract(chm,
                  buffer(vect(cbind(vegsub$adjEasting,
                  vegsub$adjNorthing)),
                  width=vegsub$adjCoordinateUncertainty),
                  fun=max)



plot(valCHMbuff$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegsub$height, pch=20, xlab="Height",
     ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(valCHMbuff$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegsub$height, use="complete")

## [1] 0.3698753

map-centric approach

In [ ]:
easting10 <- 10*floor(vegsub$adjEasting/10)

northing10 <- 10*floor(vegsub$adjNorthing/10)

vegsub <- cbind(vegsub, easting10, northing10)

In [ ]:
attributes(vegsub)

In [ ]:
vegbin <- stats::aggregate(vegsub,
                           by=list(vegsub$easting10,
                                   vegsub$northing10),
                           FUN=max)

In [ ]:
CHM10 <- terra::aggregate(chm, fact=10, fun=max)

plot(CHM10, col=topo.colors(5))

In [ ]:
vegbin$easting10 <- vegbin$easting10 + 5

vegbin$northing10 <- vegbin$northing10 + 5

binCHM <- extract(CHM10, cbind(vegbin$easting10,
                               vegbin$northing10))

plot(binCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegbin$height, pch=20,
     xlab="Height", ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(binCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegbin$height, use="complete")

## [1] 0.2244228

tree-centric approach

In [ ]:
vegsub <- vegsub[order(vegsub$height,
                       decreasing=T),]

In [ ]:
vegfil <- vegsub

for(i in 1:nrow(vegsub)) {
    if(is.na(vegfil$height[i]))
        next
    dist <- sqrt((vegsub$adjEasting[i]-vegsub$adjEasting)^2 +
                (vegsub$adjNorthing[i]-vegsub$adjNorthing)^2)
    vegfil$height[which(dist<0.3*vegsub$height[i] &
                        vegsub$height<vegsub$height[i])] <- NA
}



vegfil <- vegfil[which(!is.na(vegfil$height)),]

In [ ]:
filterCHM <- extract(chm,
                     cbind(vegfil$adjEasting,
                           vegfil$adjNorthing))

plot(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegfil$height, pch=20,
     xlab="Height", ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegfil$height)

## [1] 0.8070586

In [ ]:
vegfil <- vegfil[which(vegfil$plantStatus=="Live"),]

filterCHM <- extract(chm,
                     cbind(vegfil$adjEasting,
                           vegfil$adjNorthing))

plot(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegfil$height, pch=20,
     xlab="Height", ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegfil$height)

## [1] 0.9057883